# Deep Learning for Time Series Analysis

## Table of Contents
1. [Introduction](#Introduction)
2. [Review of Time Series Data](#Review-of-Time-Series-Data)
3. [Review of Deep Learning Concepts](#Review-of-Deep-Learning-Concepts)
4. [Environment Setup](#Environment-Setup)
5. [Data Exploration and Preparation](#Data-Exploration-and-Preparation)
6. [Constructing a Simple LSTM Model](#Constructing-a-Simple-LSTM-Model)
   - [Implementation on Dataset A](#Implementation-on-Dataset-A)
   - [Exercise on Dataset B](#Exercise-on-Dataset-B)
7. [Model Evaluation Techniques](#Model-Evaluation-Techniques)
8. [Exploring Advanced Deep Learning Models](#Exploring-Advanced-Deep-Learning-Models)
   - [CNNs](#CNNs)
   - [Transformers](#Transformers)
9. [Conclusion](#Conclusion)
10. [Additional Resources](#Additional-Resources)

## Introduction
This tutorial focuses on applying deep learning techniques to time series data. It is designed for individuals who have an introductory knowledge of both deep learning and time series analysis. The subsequent sections will delve into practical applications and enhancements of this knowledge by introducing more sophisticated deep learning architectures. 

Each concept and model discussed will be illustrated with an example. Following the example, exercises are provided to apply similar techniques on a different dataset. Draft cells with hints are included to facilitate the application of the concepts.

The aim of this guide is to enable a comprehensive understanding of the application of deep learning models to time series data, emphasizing practical skills and critical analysis of model performance.

## 2. Review of Time Series Data
Time series data consists of sequences of values recorded over intervals of time. This type of data is distinguished by its chronological order, which is crucial in analysis as the time component often carries essential structural information that can help in predicting future values. Common examples include financial market data, weather readings, and Internet of Things (IoT) sensor data. Understanding the characteristics of time series data, such as trend, seasonality, and cyclic behavior, is vital for effective modeling and forecasting.

## 3. Review of Deep Learning Concepts
Deep learning is a subset of machine learning where artificial neural networks, algorithms inspired by the human brain, learn from large amounts of data. For time series, deep learning models can learn to predict future values from past data. Key concepts in deep learning include layers, neurons, activation functions, and backpropagation. Specialized structures like Long Short-Term Memory (LSTM) units and Gated Recurrent Units (GRU) are particularly suited for time series data due to their ability to capture temporal dependencies and forget irrelevant parts of the data over time.

## 4. Environment Setup
Before diving into the models, it is necessary to set up a Python environment equipped with libraries essential for data handling and modeling. Use the following commands to install the required libraries using pip:


In [ ]:
! pip install numpy pandas matplotlib tensorflow

Ensure the Python environment is correctly set up by importing these libraries:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

## 5. Data Exploration and Preparation
This section will utilize two datasets for deep learning applications. The first dataset, from NOAA, contains hourly weather observations, and the second dataset includes daily stock prices from Google available on Kaggle.

### Loading and Visualizing the Weather Data
The NOAA dataset provides hourly weather observations that are ideal for demonstrating time series analysis.



In [ ]:
weather_df = pd.read_csv('https://www.ncei.noaa.gov/data/global-summary-of-the-day/doc/sample.csv')


In [ ]:
weather_df['DATE'] = pd.to_datetime(weather_df['DATE'])
weather_df.plot(y='TEMP', x='DATE', title='Hourly Temperature Observations')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.show()


#### Exercise 1: Loading and Visualizing the Stock Market Data
The Google stock price data will be used to apply the concepts learned from the weather data to a financial time series.
1. Load the Google stock price data from the provided CSV file.

In [ ]:
stock_url = 'https://raw.githubusercontent.com/zhemdi/Machine-learning-classes/master/GOOGL.csv'
stock_df = pd.read_csv(stock_url)

In [ ]:

stock_df.plot(y='Close', x='Date', title='Google Stock Prices Over Time')
plt.xlabel('Date')
plt.ylabel('Stock Price (USD)')
plt.show()


### Preprocessing the Data
Both datasets will undergo cleaning, normalization, and transformation into a format suitable for LSTM modeling, including sequence generation and splitting into training and testing sets.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the weather data
scaler = MinMaxScaler(feature_range=(0, 1))
weather_scaled = scaler.fit_transform(weather_df[['TEMP']])



#### Exercise 2: Data Preparation
1. Normalize the stock price data using MinMaxScaler.

In [ ]:
# Normalize the stock data
stock_scaled = scaler.fit_transform(stock_df[['Close']])

### Train-test split

In [ ]:
from sklearn.model_selection import train_test_split

# Function to create sequences
def create_sequences(data, sequence_length):
    xs, ys = [], []
    for i in range(len(data)-sequence_length):
        xs.append(data[i:i+sequence_length])
        ys.append(data[i+sequence_length])
    return np.array(xs), np.array(ys)

In [ ]:
sequence_length = 60  # Using last 60 hours/days to predict the next hour/day
weather_input, weather_target = create_sequences(weather_scaled, sequence_length)
weather_num_observations = weather_input.shape[0]
test_size = 0.2
weather_train, weather_target_train = weather_input[:int(weather_num_observations*(1-test_size))], weather_target[:int(weather_num_observations*(1-test_size))]
weather_test, weather_target_test = weather_input[int(weather_num_observations*(1-test_size)):], weather_target[int(weather_num_observations*(1-test_size)):]


#### Exercise 3: train-test split of the stock dataset

In [ ]:
stock_input, stock_target = create_sequences(stock_scaled, sequence_length)
stock_num_observations = stock_input.shape[0]
stock_train, stock_target_train = stock_input[:int(stock_num_observations*(1-test_size))], stock_target[:int(stock_num_observations*(1-test_size))]
stock_test, stock_target_test = stock_input[int(stock_num_observations*(1-test_size)):], stock_target[int(stock_num_observations*(1-test_size)):]

## 6. Constructing a Simple LSTM Model
Using the preprocessed data, LSTM models will be constructed to predict future values based on past observations.

### Implementation on Dataset A (Weather Data)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Prepare sequence data for the weather dataset...
weather_model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(sequence_length, 1)),
    # LSTM(64, return_sequences=True),
    LSTM(64),
    Dense(1)
])


In [ ]:
weather_model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model
weather_model.fit(weather_train, weather_target_train, epochs=100, batch_size=32)

#### Exercise 4: leverage the LSTM architecture to forecast future stock prices using the preprocessed Google stock data.

In [ ]:
# Prepare sequence data for the stock dataset...
stock_model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(sequence_length, 1)),
    LSTM(64),
    Dense(1)
])



In [ ]:
stock_model.compile(optimizer='adam', loss='mean_squared_error')



In [ ]:
# Train the model
stock_model.fit(stock_train, stock_target_train, epochs=20, batch_size=32)

## 7. Model Evaluation Techniques
After training the LSTM models on the weather and stock datasets, it is critical to evaluate their predictive accuracy to understand how well they are likely to perform in practical scenarios. This section will outline common evaluation techniques for time series forecasting models.


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

# Making predictions on the test set
weather_predictions = weather_model.predict(weather_test)

# Calculating RMSE and MAE
weather_rmse = mean_squared_error(weather_target_test, weather_predictions, squared=False)
weather_mae = mean_absolute_error(weather_target_test, weather_predictions)

print(f"Weather Data Test RMSE: {weather_rmse:.3f}")
print(f"Weather Data Test MAE: {weather_mae:.3f}")



In [ ]:
# Plotting actual vs predicted values
plt.figure(figsize=(10, 5))
plt.plot(weather_target_test, label='Actual')
plt.plot(weather_predictions, label='Predicted', alpha=0.7)
plt.title('Actual vs Predicted Temperature')
plt.xlabel('Time')
plt.ylabel('Normalized Temperature')
plt.legend()
plt.show()

#### Exercise 5: Evaluating the Google Stock Model

In [ ]:
# Making predictions on the test set
stock_predictions = stock_model.predict(stock_test)

# Calculating RMSE and MAE
stock_rmse = mean_squared_error(stock_target_test, stock_predictions, squared=False)
stock_mae = mean_absolute_error(stock_target_test, stock_predictions)

print(f"Google Stock Data Test RMSE: {stock_rmse:.3f}")
print(f"Google Stock Data Test MAE: {stock_mae:.3f}")

In [ ]:
# Plotting actual vs predicted values
plt.figure(figsize=(10, 5))
plt.plot(stock_target_test, label='Actual')
plt.plot(stock_predictions, label='Predicted', alpha=0.7)
plt.title('Actual vs Predicted Google Stock Prices')
plt.xlabel('Time')
plt.ylabel('Normalized Stock Price')
plt.legend()
plt.show()

## 8. Advanced Deep Learning Models
While LSTMs are highly effective for sequence prediction tasks, incorporating Convolutional Neural Networks (CNNs) can provide a different approach that exploits spatial hierarchies in data. CNNs are particularly adept at identifying patterns in data, making them useful for time series forecasting where patterns across time steps dictate future outcomes.


### CNNs
CNNs can process time series data by treating sequences as one-dimensional spatial inputs. This allows the model to capture temporal dependencies using filters and pooling layers. Here's how to implement a simple CNN for the NOAA weather dataset.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Building the CNN model
cnn_model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(weather_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])


In [ ]:
cnn_model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model
cnn_model.fit(weather_train, weather_target_train, epochs=100, batch_size=32)

#### Exercise 6: CNNs on Google Stock Data
Leverage the CNN architecture to forecast future stock prices using the Google stock data. This exercise will help solidify understanding of how CNNs can be applied to different types of time series data.

In [ ]:
# Prepare the CNN model for the stock dataset
cnn_stock_model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(stock_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])


In [ ]:
cnn_stock_model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model
cnn_stock_model.fit(stock_test, stock_target_test, epochs=100, batch_size=32)

### Transformers
Transformers can be effectively applied to time series data by treating the sequence of data points as a series of inputs that the model can attend to simultaneously. Here's how to implement a simple Transformer model for the NOAA weather dataset.


In [ ]:
import numpy as np
import tensorflow as tf

def get_positional_encoding(max_seq_len, embed_dim):
    pos_enc = np.array([
        [pos / np.power(10000, 2 * (j//2) / embed_dim) for j in range(embed_dim)]
        if pos != 0 else np.zeros(embed_dim) 
        for pos in range(max_seq_len)
        ])
    pos_enc[1:, 0::2] = np.sin(pos_enc[1:, 0::2])  # dim 2i
    pos_enc[1:, 1::2] = np.cos(pos_enc[1:, 1::2])  # dim 2i+1
    return tf.cast(pos_enc, dtype=tf.float32)

# Usage
sequence_length = 60  # or the length of your time series data
embedding_dim = 128   # or the embedding size of your model

positional_encoding = get_positional_encoding(sequence_length, embedding_dim)


In [ ]:
weather_train.shape

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, GlobalAveragePooling1D
from tensorflow.keras.models import Model

class TimeSeriesTransformer(tf.keras.layers.Layer):
    def __init__(self, input_dim, num_heads, ff_dim, max_len, rate=0.1):
        super(TimeSeriesTransformer, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=input_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(input_dim),
        ])
        self.pos_enc = get_positional_encoding(max_len, input_dim)
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        # Add batch dimension to positional encoding and repeat it to match the batch size of inputs
        pos_enc = tf.expand_dims(self.pos_enc, 0)  # shape becomes [1, seq_length, embed_dim]
        pos_enc = tf.tile(pos_enc, [tf.shape(inputs)[0], 1, 1])  # shape becomes [batch_size, seq_length, embed_dim]

        # Concatenate inputs with positional encodings
        inputs = tf.concat([inputs, pos_enc], axis=-1)  # Ensure inputs is [batch_size, seq_length, 1] before this step

        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
# Model setup
input_layer = Input(shape=(sequence_length, 1))
x = TimeSeriesTransformer(1, num_heads=8, ff_dim=256, max_len=sequence_length)(input_layer)
x = GlobalAveragePooling1D()(x)
x = Dense(1)(x)
transformer_model = Model(inputs=input_layer, outputs=x)

In [ ]:
transformer_model.compile(optimizer="adam", loss="mean_squared_error")

In [ ]:
# Train the model
transformer_model.fit(weather_train, weather_target_train, epochs=100, batch_size=32)

#### Exercise 7: Transformers on Google Stock Data

In [ ]:
# Model setup
input_layer = Input(shape=(sequence_length, 1))
x = TimeSeriesTransformer(1, num_heads=8, ff_dim=256, max_len=sequence_length)(input_layer)
x = GlobalAveragePooling1D()(x)
x = Dense(1)(x)
transformer_stock_model = Model(inputs=input_layer, outputs=x)

In [ ]:
transformer_stock_model.compile(optimizer="adam", loss="mean_squared_error")

In [ ]:
# Train the model
transformer_stock_model.fit(stock_train, stock_target_train, epochs=20, batch_size=32)

### Evaluation of CNNs and Transformers
Model evaluation is key to determining the effectiveness of CNNs and Transformers in forecasting time series data. This subsection will detail the evaluation process for both models on the NOAA weather data and Google stock data.


#### Evaluating the CNN on NOAA Weather Data
After training the CNN model on the weather data, we assess its performance using Root Mean Squared Error (RMSE) and Mean Absolute Error (MAE), as well as visual comparisons of actual vs. predicted values:


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

# Make predictions
weather_cnn_predictions = cnn_model.predict(weather_test)

# Calculate evaluation metrics
weather_cnn_rmse = mean_squared_error(weather_target_test, weather_cnn_predictions, squared=False)
weather_cnn_mae = mean_absolute_error(weather_target_test, weather_cnn_predictions)

print(f"Weather Data CNN RMSE: {weather_cnn_rmse:.3f}")
print(f"Weather Data CNN MAE: {weather_cnn_mae:.3f}")



In [ ]:
# Plot actual vs predicted
plt.figure(figsize=(10, 5))
plt.plot(weather_target_test, label='Actual')
plt.plot(weather_cnn_predictions, label='Predicted', alpha=0.7)
plt.title('Actual vs Predicted Temperature - CNN Model')
plt.xlabel('Time')
plt.ylabel('Normalized Temperature')
plt.legend()
plt.show()

#### Exercise 7: Evaluating the CNN on Google Stock Data
Similarly, evaluate the CNN model trained on the Google stock data:

In [ ]:
# Make predictions
stock_cnn_predictions = cnn_stock_model.predict(stock_test)

# Calculate evaluation metrics
stock_cnn_rmse = mean_squared_error(stock_target_test, stock_cnn_predictions, squared=False)
stock_cnn_mae = mean_absolute_error(stock_target_test, stock_cnn_predictions)

print(f"Stock Data CNN RMSE: {stock_cnn_rmse:.3f}")
print(f"Stock Data CNN MAE: {stock_cnn_mae:.3f}")



In [ ]:
# Plot actual vs predicted
plt.figure(figsize=(10, 5))
plt.plot(stock_target_test, label='Actual')
plt.plot(stock_cnn_predictions, label='Predicted', alpha=0.7)
plt.title('Actual vs Predicted Stock Prices - CNN Model')
plt.xlabel('Time')
plt.ylabel('Normalized Stock Price')
plt.legend()
plt.show()

#### Evaluating the Transformer on NOAA Weather Data
Evaluate the performance of the Transformer model on the weather dataset using similar metrics:

In [ ]:
# Make predictions
weather_transformer_predictions = transformer_model.predict(weather_test)

# Calculate evaluation metrics
weather_transformer_rmse = mean_squared_error(weather_target_test, weather_transformer_predictions, squared=False)
weather_transformer_mae = mean_absolute_error(weather_target_test, weather_transformer_predictions)

print(f"Weather Data Transformer RMSE: {weather_transformer_rmse:.3f}")
print(f"Weather Data Transformer MAE: {weather_transformer_mae:.3f}")



In [ ]:
# Plot actual vs predicted
plt.figure(figsize=(10, 5))
plt.plot(weather_target_test, label='Actual')
plt.plot(weather_transformer_predictions, label='Predicted', alpha=0.7)
plt.title('Actual vs Predicted Temperature - Transformer Model')
plt.xlabel('Time')
plt.ylabel('Normalized Temperature')
plt.legend()
plt.show()


#### Exercise 8: Evaluating the Transformer on Google Stock Data
Finally, assess how the Transformer model performs on the Google stock data:

In [ ]:
# Make predictions
stock_transformer_predictions = transformer_stock_model.predict(stock_test)

# Calculate evaluation metrics
stock_transformer_rmse = mean_squared_error(stock_target_test, stock_transformer_predictions, squared=False)
stock_transformer_mae = mean_absolute_error(stock_target_test, stock_transformer_predictions)

print(f"Stock Data Transformer RMSE: {stock_transformer_rmse:.3f}")
print(f"Stock Data Transformer MAE: {stock_transformer_mae:.3f}")



In [ ]:
# Plot actual vs predicted
plt.figure(figsize=(10, 5))
plt.plot(stock_target_test, label='Actual')
plt.plot(stock_transformer_predictions, label='Predicted', alpha=0.7)
plt.title('Actual vs Predicted Stock Prices - Transformer Model')
plt.xlabel('Time')
plt.ylabel('Normalized Stock Price')
plt.legend()
plt.show()

## Conclusion

This tutorial has systematically explored the application of deep learning methods to time series data. It began with foundational concepts in data preparation and visualization, followed by the development of various deep learning models suited for time series analysis.

### Summary of Key Points:
- **Data Preparation**: Emphasis was placed on the significance of data normalization and sequence creation, essential for adapting time series data for neural network inputs.
- **Model Development**: Techniques such as Long Short-Term Memory (LSTM) networks were detailed, highlighting their utility in capturing temporal dependencies. Further, advanced neural architectures, including Convolutional Neural Networks (CNNs) and Transformers, were adapted for time series data, exploiting their capabilities in recognizing spatial patterns and managing sequence relationships through self-attention mechanisms, respectively.
- **Evaluation Methods**: The tutorial introduced methods for assessing model performance using quantitative metrics like Root Mean Square Error (RMSE) and Mean Absolute Error (MAE), in addition to visual analysis techniques. These methods are critical for validating the effectiveness of predictive models in time series analysis.

### Recommendations for Further Study:
- **Hyperparameter Tuning**: There is substantial scope for improving model outcomes through the fine-tuning of hyperparameters such as layer configurations, neuron counts, and learning rates.
- **Integration of Model Architectures**: Exploring hybrid models that combine the strengths of different neural network types could potentially enhance predictive accuracy by capturing a broader range of data patterns.
- **Diversification of Data Sources**: Utilizing datasets from varied domains can increase the generalizability and robustness of predictive models. This diversification supports the development of models capable of handling complex, real-world datasets characterized by noise, non-stationarity, and heterogeneity.
- **Deployment in Operational Environments**: Implementing these models within real-time data processing frameworks can provide insights into their performance under dynamic conditions, highlighting potential areas for model refinement.

### Advancing Your Expertise in Deep Learning for Time Series
Deep learning for time series data is a dynamic area of research with significant implications across numerous sectors, including finance, environmental science, and healthcare. Ongoing education in the latest research methodologies and active participation in scholarly discussions are recommended to maintain a cutting-edge understanding of the field.

This tutorial aimed to provide a robust introduction to the methodologies applicable in this sphere, setting a foundation for further investigation and innovation in deep learning for time series analysis.
